In [1]:
from syft import DataSubjectList

/home/e/anaconda3/envs/Hagrid/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import numpy as np

In [3]:
from syft import DataSubjectList

In [4]:
a = np.ones((2, 1, 5))
a

array([[[1., 1., 1., 1., 1.]],

       [[1., 1., 1., 1., 1.]]])

In [5]:
b = np.ones((10, 1, 1))
b

array([[[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]],

       [[1.]]])

In [6]:
len(b)

10

(2, 1, 5) and (3, 1, 1) -> (2, 1, 5) -> (max(shape[0], shape[1]))


if len(a) < len(b) -> we have to create a bigger array
if len(a) == len(b) -> it fits!!!!
if len(a) > len(b) -> append some NaNs at the end of b
(3, 1, 5) and (2, 1, 1) -> (3, 1, 5)
ds1
ds2
NaN



(3, 50, 50) and 

In [7]:
dsl = DataSubjectList.from_objs(np.random.choice([10, 11, 12], (5, 5)))

In [8]:
dsl.one_hot_lookup, dsl.data_subjects_indexed.shape

(array([10, 11, 12]), (5, 5))

dsl1 - > (5, 5)
dsl2 -> (3, 5)
index -> 0

dsl1.data_subjects_indexed.shape -> (1, 5)
dsl2 -> (3, 5)

output -> (3, 5, 5)

INPUT:
array([[1, 0, 1, 2, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])
      
OUTPUT:
array([[6, 6, 6, 6, 6],
       [1, 0, 0, 0, 1],
       [0, 0, 2, 0, 1],
       [2, 0, 2, 1, 0],
       [2, 2, 1, 2, 0]])

array([[7, 7, 7, 7, 7],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])

array([[8, 8, 8, 8, 8],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN],
       [NAN, NAN, NAN, NAN, NAN]])



INPUT

child = [1, 2, 3]
dsl.dsi.shape != child.shape -> GammaTensor
(max(unique_data_subjects_per_data_point), 1, 3)

child = [1 2, 3
         4, 5, 6]
dsl.dsi.shape == child.shape -> PhiTensor

(0, 0, 0;
1, 1, 1)

(6, 6, 6,
7, 7, 7)

index = 0

In [61]:
def get_output_shape(shape1, shape2):
    if len(shape1) > len(shape2) or len(shape2) - len(shape1) > 1:
        print(shape1, shape2)
        raise NotImplementedError
        
    first_dim = max(shape1[0], shape2[0]) if len(shape1) == len(shape2) else shape2[0]
    other_dims = shape1 if len(shape1) < len(shape2) else shape1[1:]
    output_size = (first_dim, *other_dims)
    return output_size


def insert(dsl1: DataSubjectList, dsl2: DataSubjectList, index) -> DataSubjectList:
    
    output_shape = get_output_shape(dsl1.shape, dsl2.shape)
    
    # TODO: Broadcasting with [1] over multiple indices?
    dsl1_len = dsl1.shape[0] if len(dsl1.shape) == len(dsl2.shape) else 1
    dsl2_len = dsl2.shape[0]
    
    
    if dsl1_len < dsl2_len:
        print("if")
        # We need to create a bigger array
        bigger_array = np.full(output_shape, np.nan)
        
        # Fill current dsl1 values into this bigger_array
        # TODO: Ask Madhava if there's a smarter/vectorized way to do this
        if dsl1_len == 1:
            bigger_array[0] = dsl1.data_subjects_indexed
        else:
            for i in range(dsl1_len):
                bigger_array[i] = dsl1[i]

        output_dsl = DataSubjectList(one_hot_lookup=dsl1.one_hot_lookup, data_subjects_indexed=bigger_array)
        bigger_array[:, index] = dsl2.data_subjects_indexed.squeeze()
        return output_dsl
        
    elif dsl1_len == dsl2_len:
        print("elif")
        # it fits perfectly
        output_dsl = dsl1.copy()
        output_dsl[index] = dsl2
        return output_dsl
    else:
        print("else")
        # add some nans
        extra_nans = np.array([np.nan] * (dsl1_len - dsl2_len) * np.prod(dsl2.shape[1:])).reshape((dsl1_len - dsl2_len, *dsl2.shape[1:]))
        array_to_append = np.append(dsl2.data_subjects_indexed, extra_nans)
        
        output_dsl = dsl1.copy()
        output_dsl[index] = array_to_append
        return output_dsl
        

In [10]:
len(dsl)

5

In [11]:
dsl.one_hot_lookup, dsl.data_subjects_indexed.shape

(array([10, 11, 12]), (5, 5))

In [12]:
dsl2 = DataSubjectList.from_objs(np.random.choice([50, 60, 70], size=(2, 1, 5)))
dsl2.one_hot_lookup, dsl2.data_subjects_indexed.shape 

(array([50, 60, 70]), (2, 1, 5))

In [13]:
output_dsl = insert(dsl1=dsl, dsl2=dsl2, index=0)

In [14]:
output_dsl

array([[[ 1.,  1.,  1.,  2.,  2.],
        [ 0.,  2.,  0.,  2.,  2.],
        [ 2.,  1.,  0.,  2.,  1.],
        [ 2.,  0.,  0.,  2.,  2.],
        [ 1.,  1.,  1.,  1.,  0.]],

       [[ 2.,  2.,  0.,  1.,  0.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [15]:
dsl[0].shape

(5,)

In [16]:
big_array = np.empty((2,5,5))
big_array

array([[[4.64495799e-310, 0.00000000e+000, 1.39387552e-258,
         6.01347002e-154, 5.50438989e+257],
        [9.18149118e+170, 3.17095857e+180, 2.58686482e+161,
         6.01718289e+175, 2.41623980e+198],
        [5.29274285e+199, 4.94142262e+257, 5.98182169e-154,
         1.72965980e+156, 6.13357581e-095],
        [1.10640382e+200, 4.73784579e+164, 5.30482732e+180,
         3.17095864e+180, 2.44085715e-154],
        [1.79805224e+044, 3.94655105e+180, 7.03542624e+199,
         1.10181113e+155, 9.77135008e-153]],

       [[2.86530675e+161, 2.28569998e-152, 7.39678251e+247,
         6.01347002e-154, 4.72883470e+164],
        [3.28097882e-085, 1.14489732e+243, 1.50818256e+161,
         5.11495729e+199, 4.24664805e+175],
        [4.26137297e+257, 6.01346953e-154, 1.27734658e-152,
         5.99265804e+175, 1.69593624e-152],
        [4.78111609e+180, 5.46090425e+241, 1.81450395e-152,
         2.86752279e+161, 1.87650139e-152],
        [6.01347002e-154, 1.12179665e+224, 1.02584502e+200,
  

In [17]:
big_array[0] = dsl.data_subjects_indexed

In [18]:
big_array

array([[[0.00000000e+000, 0.00000000e+000, 1.00000000e+000,
         0.00000000e+000, 1.00000000e+000],
        [0.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [2.00000000e+000, 1.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 1.00000000e+000],
        [2.00000000e+000, 0.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         1.00000000e+000, 0.00000000e+000]],

       [[2.86530675e+161, 2.28569998e-152, 7.39678251e+247,
         6.01347002e-154, 4.72883470e+164],
        [3.28097882e-085, 1.14489732e+243, 1.50818256e+161,
         5.11495729e+199, 4.24664805e+175],
        [4.26137297e+257, 6.01346953e-154, 1.27734658e-152,
         5.99265804e+175, 1.69593624e-152],
        [4.78111609e+180, 5.46090425e+241, 1.81450395e-152,
         2.86752279e+161, 1.87650139e-152],
        [6.01347002e-154, 1.12179665e+224, 1.02584502e+200,
  

In [19]:
dsl2.data_subjects_indexed

array([[[1, 1, 1, 2, 2]],

       [[2, 2, 0, 1, 0]]])

In [20]:
big_array[:, 0, :].shape

(2, 5)

In [21]:
big_array[:, 0, :] = dsl2.data_subjects_indexed.squeeze()

In [22]:
big_array

array([[[1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [0.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [2.00000000e+000, 1.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 1.00000000e+000],
        [2.00000000e+000, 0.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         1.00000000e+000, 0.00000000e+000]],

       [[2.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         1.00000000e+000, 0.00000000e+000],
        [3.28097882e-085, 1.14489732e+243, 1.50818256e+161,
         5.11495729e+199, 4.24664805e+175],
        [4.26137297e+257, 6.01346953e-154, 1.27734658e-152,
         5.99265804e+175, 1.69593624e-152],
        [4.78111609e+180, 5.46090425e+241, 1.81450395e-152,
         2.86752279e+161, 1.87650139e-152],
        [6.01347002e-154, 1.12179665e+224, 1.02584502e+200,
  

In [23]:
index = 0
big_array[:, index].shape

(2, 5)

In [24]:
index = [0, 1]
big_array[:, index, :]

array([[[1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [0.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000]],

       [[2.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         1.00000000e+000, 0.00000000e+000],
        [3.28097882e-085, 1.14489732e+243, 1.50818256e+161,
         5.11495729e+199, 4.24664805e+175]]])

In [25]:
big_array

array([[[1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [0.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [2.00000000e+000, 1.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 1.00000000e+000],
        [2.00000000e+000, 0.00000000e+000, 0.00000000e+000,
         2.00000000e+000, 2.00000000e+000],
        [1.00000000e+000, 1.00000000e+000, 1.00000000e+000,
         1.00000000e+000, 0.00000000e+000]],

       [[2.00000000e+000, 2.00000000e+000, 0.00000000e+000,
         1.00000000e+000, 0.00000000e+000],
        [3.28097882e-085, 1.14489732e+243, 1.50818256e+161,
         5.11495729e+199, 4.24664805e+175],
        [4.26137297e+257, 6.01346953e-154, 1.27734658e-152,
         5.99265804e+175, 1.69593624e-152],
        [4.78111609e+180, 5.46090425e+241, 1.81450395e-152,
         2.86752279e+161, 1.87650139e-152],
        [6.01347002e-154, 1.12179665e+224, 1.02584502e+200,
  

In [26]:
arr = np.ones((1, 1, 5))

In [27]:
arr.squeeze().shape

(5,)

In [28]:
dsl[0].data_subjects_indexed.shape

(5,)

In [29]:
test = dsl.copy()

In [30]:
test.data_subjects_indexed

array([[0, 0, 1, 0, 1],
       [0, 2, 0, 2, 2],
       [2, 1, 0, 2, 1],
       [2, 0, 0, 2, 2],
       [1, 1, 1, 1, 0]])

In [31]:
sq =arr.squeeze()
test.data_subjects_indexed[0] = sq

In [32]:
test.data_subjects_indexed

array([[1, 1, 1, 1, 1],
       [0, 2, 0, 2, 2],
       [2, 1, 0, 2, 1],
       [2, 0, 0, 2, 2],
       [1, 1, 1, 1, 0]])

In [33]:
sq

array([1., 1., 1., 1., 1.])

In [34]:
dsl2 = DataSubjectList.from_objs(np.random.choice([50, 60, 70], size=(1, 1, 5)))

In [35]:
output_dsl = insert(dsl1=dsl, dsl2=dsl2, index=0)

In [36]:
output_dsl.data_subjects_indexed

array([[3, 4, 4, 4, 4],
       [0, 2, 0, 2, 2],
       [2, 1, 0, 2, 1],
       [2, 0, 0, 2, 2],
       [1, 1, 1, 1, 0]])

In [39]:
output_dsl.one_hot_lookup

array([10, 11, 12, 60, 70])

In [37]:
dsl2.data_subjects_indexed

array([[[0, 1, 1, 1, 1]]])

In [38]:
dsl2.one_hot_lookup

array([60, 70])

In [41]:
arr2 = np.ones((5,5)).squeeze()

In [42]:
arr2.shape

(5, 5)

In [62]:
def test(dsl1_shape, dsl2_shape, index) -> DataSubjectList:
    dsl1 = DataSubjectList.from_objs(np.random.choice([80, 90, 100], size=dsl1_shape))
    dsl2 = DataSubjectList.from_objs(np.random.choice([50, 60, 70], size=dsl2_shape))
    print(dsl1.one_hot_lookup, dsl1.data_subjects_indexed.shape)
    print(dsl2.one_hot_lookup, dsl2.data_subjects_indexed.shape)
    
    output_dsl = insert(dsl1=dsl, dsl2=dsl2, index=index)
    return output_dsl

In [63]:
res = test((5, 5), (3, 1, 5), index=0)

[ 80  90 100] (5, 5)
[50 60 70] (3, 1, 5)
if


In [64]:
res.data_subjects_indexed

array([[[ 1.,  0.,  0.,  2.,  0.],
        [ 0.,  2.,  0.,  2.,  2.],
        [ 2.,  1.,  0.,  2.,  1.],
        [ 2.,  0.,  0.,  2.,  2.],
        [ 1.,  1.,  1.,  1.,  0.]],

       [[ 2.,  0.,  1.,  1.,  2.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[ 1.,  1.,  1.,  0.,  2.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [65]:
res = test((5, 5), (3, 1, 5), index=2)

[ 80  90 100] (5, 5)
[50 60 70] (3, 1, 5)
if


In [66]:
res.data_subjects_indexed

array([[[ 0.,  0.,  1.,  0.,  1.],
        [ 0.,  2.,  0.,  2.,  2.],
        [ 2.,  1.,  2.,  1.,  2.],
        [ 2.,  0.,  0.,  2.,  2.],
        [ 1.,  1.,  1.,  1.,  0.]],

       [[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [ 2.,  2.,  2.,  0.,  0.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]],

       [[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [ 1.,  0.,  0.,  1.,  0.],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]])

In [68]:
res = test((5, 5), (3, 1, 5), index=-1)

[ 80  90 100] (5, 5)
[50 60 70] (3, 1, 5)
if


In [69]:
res.data_subjects_indexed

array([[[ 0.,  0.,  1.,  0.,  1.],
        [ 0.,  2.,  0.,  2.,  2.],
        [ 2.,  1.,  0.,  2.,  1.],
        [ 2.,  0.,  0.,  2.,  2.],
        [ 0.,  2.,  2.,  1.,  0.]],

       [[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [ 2.,  0.,  1.,  2.,  2.]],

       [[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [ 2.,  0.,  2.,  0.,  2.]]])